# SciPost Sample Code

* Author: Pochung Chen
* Last update: 2024/6/20

In [1]:
import cytnx
import numpy as np
print(cytnx.__file__)

/Users/pcchen/myvenv/lib/python3.13/site-packages/cytnx-1.0.0-py3.13-macosx-15.0-arm64.egg/cytnx/__init__.py


## Introduction
### Sneak preview: tensors and tensor contraction

<img src="figs/tensor_diagrams.png" alt="drawing" width="400px"/>
<img src="figs/mps_tensor.png" alt="drawing" width="200px"/>

In [2]:
# checked
A = cytnx.UniTensor.ones([8,2,8]).relabels(["v1","phy","v2"])\
                         .set_name("A")
# or, using argument names:
A = cytnx.UniTensor.ones([8,2,8], labels=["v1","phy","v2"], name="A")

<img src="figs/contraction_ABC.png" alt="drawing" width="300px"/>

In [3]:
# checked, modified
# Initialize UniTensors and their labels
A = cytnx.UniTensor.ones([2,2,2])\
                       .set_name("A").relabel(["alpha","beta","gamma"])
B = cytnx.UniTensor.ones([2,2])\
                       .set_name("B").relabel(["beta","delta"])
C = cytnx.UniTensor.ones([2])\
                       .set_name("C").relabel(["gamma"])
# Do the contractions
# Do the contractions
AB = cytnx.Contract([A, B])
D = cytnx.Contract([AB, C])
# Similarly, in one step:
D = cytnx.Contract([A,B,C])

<img src="figs/matrix_multiplication.png" alt="drawing" width="400px"/>

In [4]:
# checked
net = cytnx.Network()
net.FromString(["M1:  i, j",
                "M2:  j, k",
                "TOUT: i; k",
                "ORDER: "])

In [5]:
# chcked
# Initialize the tensors and their labels
A = cytnx.UniTensor.ones([2,2]).relabel(["a","b"]).set_name("A")
B = cytnx.UniTensor.ones([2,2]).relabel(["c","d"]).set_name("B")
# Put the tensors into the network
net.PutUniTensor("M1", A, ["a", "b"])
net.PutUniTensor("M2", B, ["c", "d"])
# Launch the contraction
AB = net.Launch()
AB.print_diagram()

-----------------------
tensor Name : 
tensor Rank : 2
block_form  : False
is_diag     : False
on device   : cytnx device: CPU
          ---------     
         /         \    
   i ____| 2     2 |____ k
         \         /    
          ---------     


In [6]:
# checked
net.PutUniTensor("M1", B, ["c", "d"])
net.PutUniTensor("M2", A, ["a", "b"])
BA = net.Launch()
BA.print_diagram()

-----------------------
tensor Name : 
tensor Rank : 2
block_form  : False
is_diag     : False
on device   : cytnx device: CPU
          ---------     
         /         \    
   i ____| 2     2 |____ k
         \         /    
          ---------     


### Conventions and object behavior
#### Object behavior

In [7]:
# checked
# Python:
A = cytnx.UniTensor.zeros([2,3])
B = A
print(B is A) # Output: True

True


In [8]:
# checked
# Python
A = cytnx.UniTensor.zeros([2,3])
B = A
C = A.clone()
print(B is A) # Output: True 
print(C is A) # Output: False

True
False


In [9]:
# checked
# Python
A = cytnx.UniTensor.zeros([2,3])
B = A.permute([1,0]) # A and B share tensor elements, different metadata
B[0,1] = 1.
C = A.clone().permute([1,0]) # C is independent of A
C[0,1] = 2.
print(A) # Original shape, element A[1,0] == 1.
print(B) # Permuted shape, element B[0,1] == 1.
print(C) # Permuted shape, element C[0,1] == 2.
D = A.permute_([1,0])   # D is a reference to A
print(A) # Permuted shape, element A[0,1] == 1.
print(D is A) # Output: True 

-------- start of print ---------
Tensor name: 
is_diag    : False
contiguous : True

Total elem: 6
type  : Double (Float64)
cytnx device: CPU
Shape : (2,3)
[[0.00000e+00 0.00000e+00 0.00000e+00 ]
 [1.00000e+00 0.00000e+00 0.00000e+00 ]]




-------- start of print ---------
Tensor name: 
is_diag    : False
contiguous : False

Total elem: 6
type  : Double (Float64)
cytnx device: CPU
Shape : (3,2)
[[0.00000e+00 1.00000e+00 ]
 [0.00000e+00 0.00000e+00 ]
 [0.00000e+00 0.00000e+00 ]]




-------- start of print ---------
Tensor name: 
is_diag    : False
contiguous : False

Total elem: 6
type  : Double (Float64)
cytnx device: CPU
Shape : (3,2)
[[0.00000e+00 2.00000e+00 ]
 [0.00000e+00 0.00000e+00 ]
 [0.00000e+00 0.00000e+00 ]]




-------- start of print ---------
Tensor name: 
is_diag    : False
contiguous : False

Total elem: 6
type  : Double (Float64)
cytnx device: CPU
Shape : (3,2)
[[0.00000e+00 1.00000e+00 ]
 [0.00000e+00 0.00000e+00 ]
 [0.00000e+00 0.00000e+00 ]]




False


## Non-symmetric Bonds

In [10]:
# checked
# Combine two bonds to create a new bond
bond1 = cytnx.Bond(10)
bond2 = cytnx.Bond(2)
# Combine bond1 and bond2
bond12 = bond1.combineBond(bond2)
print(bond1)    # Output: Dim = 10 |type: REGULAR
print(bond2)    # Output: Dim = 2 |type: REGULAR
print(bond12)   # Output: Dim = 20 |type: REGULAR

Dim = 10 | type: REGULAR 


Dim = 2 | type: REGULAR 


Dim = 20 | type: REGULAR 




In [11]:
# checked
# Merge one bond into another bond
bond1 = cytnx.Bond(10)
bond2 = cytnx.Bond(2)
# Merge bond2 into bond1
bond1.combineBond_(bond2)
print(bond1)    # Dim = 20 |type: REGULAR

Dim = 20 | type: REGULAR 




In [12]:
# checked
# Merge multiple bonds
bond1 = cytnx.Bond(2)
bond2 = cytnx.Bond(2)
bond3 = cytnx.Bond(2)
# Merge bond2 and bond 3 with bond1
bond123 = bond1.combineBonds([bond2,bond3])
print(bond123)  # Output: Dim = 8 |type: REGULAR
bond1.combineBonds_([bond2,bond3])
print(bond1)    # Output: Dim = 8 |type: REGULAR

Dim = 8 | type: REGULAR 


Dim = 8 | type: REGULAR 




## Non-Symmteric UniTensor
<img src="figs/uniTensor_schematic.png" alt="drawing" width="500px"/>

### Creating a non-symmetric UniTensor
<img src="figs/tensor_diagram.png" alt="drawing" width="300px"/>

In [62]:
# checked, modified
# Create a \lstinline{UniTensor} by using generators
# Create a rank-3 UniTensor with shape [2,3,4]
uT = cytnx.UniTensor.arange(2*3*4).reshape(2,3,4)\
                            .relabel(["a","b","c"])\
                            .set_name("tensor uT")
uT.print_diagram()

-----------------------
tensor Name : tensor uT
tensor Rank : 3
block_form  : False
is_diag     : False
on device   : cytnx device: CPU
         --------     
        /        \    
        |      2 |____ a
        |        |    
        |      3 |____ b
        |        |    
        |      4 |____ c
        \        /    
         --------     


In [14]:
# checked, modified
# Create a \lstinline{UniTensor} from \lstinline{Bond} objects
# Define bonds by their dimensions
bond1 = cytnx.Bond(2)
bond2 = cytnx.Bond(3)
bond3 = cytnx.Bond(4)
# Create a UniTensor by the bonds
uT2 = cytnx.UniTensor([bond1,bond2,bond3])\
                      .relabel(["a","b","c"]).set_name("uT2")
# or, alternative initialization:
uT3 = cytnx.UniTensor([bond1,bond2,bond3], labels=["a","b","c"],\
                      name="uT3")
# Get a bond from a UniTensor
bonda = uT2.bond("a")

In [15]:
# checked, modified
# Create a \lstinline{UniTensor} with complex elements
# 1. Create a complex valued UniTensor by initializer
uTc = cytnx.UniTensor.zeros([2,2], dtype=cytnx.Type.ComplexDouble)\
                            .relabel(["a","b"]).set_name("uT complex")
# 2. Create a complex valued UniTensor by Bond
bond1 = cytnx.Bond(2)
bond2 = cytnx.Bond(2)
uTc2 = cytnx.UniTensor([bond1,bond2], dtype=cytnx.Type.ComplexDouble)\
                       .relabel(["bd1","bd2"]).set_name("uT complex 2")

In [16]:
# checked
# Create \lstinline{UniTensor} with random elements
# Create a random tensor with elements uniformly in the range [0, 1]
rT = cytnx.UniTensor.uniform([2,3,2], low=0., high=1.)
# Create a random tensor with elements from a Gaussian distribution
# with mean value 0 and standard deviation 1
nT = cytnx.UniTensor.normal([2,3,2], mean=0., std=1.)

In [17]:
# chcked
# Randomize the elements of a \lstinline{UniTensor}
# Supposes uT is a UniTensor that is already created as in (*\color{codegreen}\cref{code:UniTensorCreation}*)
# Randomize the elements of uT uniformly in the range [-1, 1]
cytnx.random.uniform_(uT, low=-1., high=1.)
# Randomize the elements of uT according to a normal distribution
# with mean value 2 and standard deviation 3
cytnx.random.normal_(uT, mean=2., std=3.)

### Viewing/Changing labels

In [18]:
# chcked, modified
# Relabel the bond(s) in a \lstinline{UniTensor}.
# Create a rank-4 UniTensor with shape [2,3,4,5]
uT = cytnx.UniTensor.ones([2,3,4,5])\
                          .relabel(["a","b","c","d"]).set_name("uT")
print(uT.labels()) # Output: ['a', 'b', 'c', 'd']                     
# Change the label "a" to "i"
uT.relabel_(old_label="a", new_label="i")
print(uT.labels()) # Output: ['i', 'b', 'c', 'd']                     
# Change multiple labels
uT.relabel_(["b","c"], ["j","k"])
print(uT.labels()) # Output: ['i', 'j', 'k', 'd']

['a', 'b', 'c', 'd']
['i', 'b', 'c', 'd']
['i', 'j', 'k', 'd']


In [19]:
# checked
uT_new = uT.relabel(old_label="d", new_label="l")
print(uT_new.labels())
print(uT_new.same_data(uT)) # Output: True

['i', 'j', 'k', 'l']
True


### Permute

In [20]:
# chcked
# Permute the indices of a \lstinline{UniTensor}
# Create a UniTensor
uT = cytnx.UniTensor.arange(2*3*4).reshape(2,3,4)\
                            .relabels(["a","b","c"]).set_name("uT")
print(uT.labels())  # Output: ['a', 'b', 'c']
print(uT.shape())   # Output: [2, 3, 4]
# Permute by index labels
uT2 = uT.permute(["b","c","a"])
print(uT2.labels()) # Output: ['b', 'c', 'a']
print(uT2.shape())  # Output: [3, 4, 2]
# Permute by index positions
uT3 = uT.permute([2,0,1])
print(uT3.labels()) # Output: ['c', 'a', 'b']
print(uT3.shape())  # Output: [4, 2, 3]

['a', 'b', 'c']
[2, 3, 4]
['b', 'c', 'a']
[3, 4, 2]
['c', 'a', 'b']
[4, 2, 3]


### Tensor information

In [21]:
uT = cytnx.UniTensor.arange(2*3*4).reshape(2,3,4)\
                            .relabel(["a","b","c"])\
                            .set_name("tensor uT")
# Print the diagram of a UniTensor
uT.print_diagram()
print(uT)

-----------------------
tensor Name : tensor uT
tensor Rank : 3
block_form  : False
is_diag     : False
on device   : cytnx device: CPU
         --------     
        /        \    
        |      2 |____ a
        |        |    
        |      3 |____ b
        |        |    
        |      4 |____ c
        \        /    
         --------     
-------- start of print ---------
Tensor name: tensor uT
is_diag    : False
contiguous : True

Total elem: 24
type  : Double (Float64)
cytnx device: CPU
Shape : (2,3,4)
[[[0.00000e+00 1.00000e+00 2.00000e+00 3.00000e+00 ]
  [4.00000e+00 5.00000e+00 6.00000e+00 7.00000e+00 ]
  [8.00000e+00 9.00000e+00 1.00000e+01 1.10000e+01 ]]
 [[1.20000e+01 1.30000e+01 1.40000e+01 1.50000e+01 ]
  [1.60000e+01 1.70000e+01 1.80000e+01 1.90000e+01 ]
  [2.00000e+01 2.10000e+01 2.20000e+01 2.30000e+01 ]]]






### Arithmetic operations and tensor operations

In [22]:
# Create a UniTensor
uTa = cytnx.UniTensor.ones([2,3]) # all the elements of uTa equal 1.0
uTb = uTa*3 + 2 # all the elements of uTb equal 5.0
uTc = uTa/uTb # all the elements of uTc equal 0.2

### Getting/Setting elements

#### UniTensor.at()

In [23]:
# checked, modified
# Get or set a single element
# Create a UniTensor
uT = cytnx.UniTensor.zeros([3,3,3]).relabel(["a","b","c"]).set_name("uT")
# Print the element of the UniTensor
print(uT.at(["a","b","c"],[0,1,2]).value) # Output: 0.0
# Set the element to a new value and see the change
uT.at(["a","b","c"], [0,1,2]).value = -1
print(uT.at(["a","b","c"],[0,1,2]).value) # Output: -1.0
uT.at(["b","a","c"], [1,0,2]).value = -2
print(uT.at(["a","b","c"],[0,1,2]).value) # Output: -2.0
uT.at(["c","b","a"], [2,1,0]).value = -3
print(uT.at(["a","b","c"],[0,1,2]).value) # Output: -3.0)

0.0
-1.0
-2.0
-3.0


In [24]:
uT.print_diagram()

-----------------------
tensor Name : uT
tensor Rank : 3
block_form  : False
is_diag     : False
on device   : cytnx device: CPU
          ---------     
         /         \    
   a ____| 3     3 |____ b
         |         |    
         |       3 |____ c
         \         /    
          ---------     


#### Slicing

In [25]:
# checked, modified
# Get or set a slice
# Create a UniTensor
uT=cytnx.UniTensor.zeros([2,3,4]).set_name("uT")\
                         .relabel(["a", "b", "c"])
# set and get an element
uT[1,1,2] = 3.0
print(uT[1,1,2])    # Output: 3.0
# get a slice of a UniTensor
uTpart = uT[0,:,1:3]
# set a slice of a UniTensor
uT[0,:,1:3]=cytnx.UniTensor.ones([3,2])
# print the original slice
print(uTpart)
# print the UniTensor
print(uT)

-------- start of print ---------
Tensor name: 
is_diag    : False
contiguous : True

Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1)
[3.00000e+00 ]




-------- start of print ---------
Tensor name: 
is_diag    : False
contiguous : True

Total elem: 6
type  : Double (Float64)
cytnx device: CPU
Shape : (3,2)
[[0.00000e+00 0.00000e+00 ]
 [0.00000e+00 0.00000e+00 ]
 [0.00000e+00 0.00000e+00 ]]




-------- start of print ---------
Tensor name: uT
is_diag    : False
contiguous : True

Total elem: 24
type  : Double (Float64)
cytnx device: CPU
Shape : (2,3,4)
[[[0.00000e+00 1.00000e+00 1.00000e+00 0.00000e+00 ]
  [0.00000e+00 1.00000e+00 1.00000e+00 0.00000e+00 ]
  [0.00000e+00 1.00000e+00 1.00000e+00 0.00000e+00 ]]
 [[0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 ]
  [0.00000e+00 0.00000e+00 3.00000e+00 0.00000e+00 ]
  [0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 ]]]






### Accessing  data in a block

#### Getting a block

In [26]:
# checked
# Get a block (\lstinline{Tensor}) from a non-symmetric \lstinline{UniTensor}
uT = cytnx.UniTensor.ones([3,3]).set_name("uT")
B = uT.get_block_()
B[0,0] = 0.
print(uT)

-------- start of print ---------
Tensor name: uT
is_diag    : False
contiguous : True

Total elem: 9
type  : Double (Float64)
cytnx device: CPU
Shape : (3,3)
[[0.00000e+00 1.00000e+00 1.00000e+00 ]
 [1.00000e+00 1.00000e+00 1.00000e+00 ]
 [1.00000e+00 1.00000e+00 1.00000e+00 ]]






#### Putting a block

In [27]:
# checked
# Replace a block of a \lstinline{UniTensor} by a \lstinline{Tensor}.
uT = cytnx.UniTensor.ones([3,3]).set_name("uT")
B = uT.get_block()
# Manipulate the Tensor
B[0,0] = 0.
# Put the Tensor back to the UniTensor
uT.put_block(B)
print(uT)

-------- start of print ---------
Tensor name: uT
is_diag    : False
contiguous : True

Total elem: 9
type  : Double (Float64)
cytnx device: CPU
Shape : (3,3)
[[0.00000e+00 1.00000e+00 1.00000e+00 ]
 [1.00000e+00 1.00000e+00 1.00000e+00 ]
 [1.00000e+00 1.00000e+00 1.00000e+00 ]]






### Converting to/from a NumPy array

In [28]:
# checked, modified
# Create the tensor of a matrix-product-operator for the transverse-field Ising model as a NumPy array and then convert it to a UniTensor.
import numpy as np
# Define operators
Sx = np.array([[0.,1.],[1.,0.]])
Sz = np.array([[1.,0.],[0.,-1.]])
I  = np.array([[1.,0.],[0.,1.]])
# Define MPO tensor
hx, hz = 0.5, 0.5       # longitudinal and transverse fields
M = np.zeros((3,3,2,2)) # The first two indices are virtual indices,
                        # and the last two are physical indices
M[0,0] = I
M[2,2] = I
M[1,0] = Sx
M[2,1] = -Sx
M[2,0] = -hz*Sz - hx*Sx
M = cytnx.from_numpy(M) # NumPy array to a Tensor (see (*\color{codegreen}\cref{sec:numpy}*))
M = cytnx.UniTensor (M) # Tensor to a UniTensor
M.set_name("M").relabel_(["mpoL", "mpoR", "physKet", "physBra"])
M.print_diagram()       # Check diagram
# Create a NumPy array from a non-symmetric UniTensor
T = M.get_block().numpy()

-----------------------
tensor Name : M
tensor Rank : 4
block_form  : False
is_diag     : False
on device   : cytnx device: CPU
             ---------     
            /         \    
   mpoL ____| 3     2 |____ physKet
            |         |    
   mpoR ____| 3     2 |____ physBra
            \         /    
             ---------     


## UniTensor with Symmetries

### Symmetry and quantum numbers

In [29]:
# checked
# Create a $U(1)$ \lstinline{Symmetry} object.
U1 = cytnx.Symmetry.U1()
print(U1)

--------------------
[Symmetry]
type : Abelian, U1
combine rule : Q1 + Q2
reverse rule : Q*(-1) 
--------------------




In [30]:
# checked
# Create a $Z_3$ \lstinline{Symmetry} object
Z3 = cytnx.Symmetry.Zn(3)
print(Z3)

--------------------
[Symmetry]
type : Abelian, Z(3)
combine rule : (Q1 + Q2)%3
reverse rule : Q*(-1) 
--------------------




### Bond with quantum numbers 

#### Creating bonds with quantum numbers

In [31]:
# checked
# Create a bond with a $U(1)$ quantum number.
#Creating a Bond from a list of quantum numbers and a list of dimensions
bond = cytnx.Bond(cytnx.BD_IN, [[2],[4]], [3,5], [cytnx.Symmetry.U1()])
#Creating the same Bond from a tuple of quantum numbers and dimensions
bond = cytnx.Bond(cytnx.BD_IN, [([2],3),([4],5)], [cytnx.Symmetry.U1()])
#Creating the same Bond with the helper function Qs
bond = cytnx.Bond(cytnx.BD_IN,\
                  [cytnx.Qs(2)>>3,cytnx.Qs(4)>>5],[cytnx.Symmetry.U1()])

In [32]:
# checked
# Create a bond with two quantum numbers
# This creates an incoming Bond
# with U(1)xZ2 quantum numbers qn_U1 and qn_Z2.
# The sector with qn_U1=2 and qn_Z2=0 has dimension 3.
# The sector with qn_U1=4 and qn_Z2=1 has dimension 5.
bond = cytnx.Bond(cytnx.BD_IN,\
                  [cytnx.Qs(2,0)>>3, cytnx.Qs(4,1)>>5],\
                  [cytnx.Symmetry.U1(), cytnx.Symmetry.Zn(2)])
print(bond)

Dim = 8 | type: | IN (KET)> 
 U1::   +2  +4
 Z2::   +0  +1
Deg>>    3   5




#### Combining bonds with quantum numbers

In [33]:
# checked
# Combine bonds with quantum numbers.
bond1 = cytnx.Bond(cytnx.BD_IN,\
                [cytnx.Qs(0)>>1,cytnx.Qs(1)>>1], [cytnx.Symmetry.U1()])
bond2 = cytnx.Bond(cytnx.BD_IN,\
                [cytnx.Qs(2)>>1,cytnx.Qs(3)>>1], [cytnx.Symmetry.U1()])
bond12 = bond1.combineBond(bond2)
print('bond1:\n',bond1)
print('bond2:\n',bond2)
print('bond12:\n',bond12)

bond1:
 Dim = 2 | type: | IN (KET)> 
 U1::   +0  +1
Deg>>    1   1


bond2:
 Dim = 2 | type: | IN (KET)> 
 U1::   +2  +3
Deg>>    1   1


bond12:
 Dim = 4 | type: | IN (KET)> 
 U1::   +2  +3  +4
Deg>>    1   2   1




In [34]:
# checked
# is_grp=False
# Combine bonds with quantum numbers.
bond1 = cytnx.Bond(cytnx.BD_IN,\
                [cytnx.Qs(0)>>1,cytnx.Qs(1)>>1], [cytnx.Symmetry.U1()])
bond2 = cytnx.Bond(cytnx.BD_IN,\
                [cytnx.Qs(2)>>1,cytnx.Qs(3)>>1], [cytnx.Symmetry.U1()])
bond12 = bond1.combineBond(bond2, is_grp=False)
print('bond1:\n',bond1)
print('bond2:\n',bond2)
print('bond12:\n',bond12)

bond1:
 Dim = 2 | type: | IN (KET)> 
 U1::   +0  +1
Deg>>    1   1


bond2:
 Dim = 2 | type: | IN (KET)> 
 U1::   +2  +3
Deg>>    1   1


bond12:
 Dim = 4 | type: | IN (KET)> 
 U1::   +2  +3  +3  +4
Deg>>    1   1   1   1





# Cytnx warning occur at void cytnx::Bond_impl::combineBond_(const boost::intrusive_ptr<cytnx::Bond_impl>&, const bool&)
# warning: [WARNING] duplicated qnums might appears!

# file : /Users/pcchen/github/Cytnx/src/Bond.cpp (268)


#### Redirecting bonds

In [35]:
# checked
# Redirect a \lstinline{Bond}.}
# Supposes bond is a Bond that is already created as in (*\color{codegreen}\cref{code:BondU1Z2}*)
print(bond)     # Output: Dim = 8 |type: |IN (KET)>
# Create a new bond that has the direction changed.
bond2 = bond.redirect()
# Change the direction of a bond
bond.redirect_()
print(bond)     # Output: Dim = 8 |type: < OUT (BRA)|
print(bond2)    # Output: Dim = 8 |type: < OUT (BRA)|

Dim = 8 | type: | IN (KET)> 
 U1::   +2  +4
 Z2::   +0  +1
Deg>>    3   5


Dim = 8 | type: <OUT (BRA)| 
 U1::   +2  +4
 Z2::   +0  +1
Deg>>    3   5


Dim = 8 | type: <OUT (BRA)| 
 U1::   +2  +4
 Z2::   +0  +1
Deg>>    3   5




In [36]:
# Define the bonds with quantum numbers
bond1 = cytnx.Bond(cytnx.BD_IN,\
                [cytnx.Qs(1)>>1, cytnx.Qs(-1)>>1],[cytnx.Symmetry.U1()])
bond2 = cytnx.Bond(cytnx.BD_IN,\
                [cytnx.Qs(1)>>1, cytnx.Qs(-1)>>1],[cytnx.Symmetry.U1()])
bond3 = cytnx.Bond(cytnx.BD_OUT,\
                [cytnx.Qs(2)>>1, cytnx.Qs(0)>>2, cytnx.Qs(-2)>>1],\
                [cytnx.Symmetry.U1()])
# Create a symmetric UniTensor with these bonds
uTsym = cytnx.UniTensor([bond1, bond2, bond3]).relabel(["a","b","c"])\
                        .set_name("uTsym").set_rowrank(0)
uTsym.print_diagram()

-----------------------
tensor Name : uTsym
tensor Rank : 3
contiguous  : True
valid blocks : 4
is diag   : False
on device   : cytnx device: CPU
     row          col 
        ----------    
        |        |    
        |      2 |<--* a
        |        |    
        |      2 |<--* b
        |        |    
        |      4 |-->  c
        |        |    
        ----------    



In [37]:
uTsym.print_blocks()

-------- start of print ---------
Tensor name: uTsym
braket_form : False
is_diag    : False
[OVERALL] contiguous : True
BLOCK [#0]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
        ----------
        |        |
        |      1 |<--* [0] U1(1)
        |        |
        |      1 |<--* [0] U1(1)
        |        |
        |      1 |-->  [0] U1(2)
        |        |
        ----------

Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,1)
[[[0.00000e+00 ]]]

BLOCK [#1]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
        ----------
        |        |
        |      1 |<--* [0] U1(1)
        |        |
        |      1 |<--* [1] U1(-1)
        |        |
        |      2 |-->  [1] U1(0)
        |        |
        ----------

Total elem: 2
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,2)
[[[0.00000e+00 0.00000e+00 ]]]

BLOCK [#2]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
        ----------
        |        |
  

In [38]:
uTsym_dag=uTsym.Dagger()
uTsym_dag.print_diagram()

-----------------------
tensor Name : uTsym
tensor Rank : 3
contiguous  : True
valid blocks : 4
is diag   : False
on device   : cytnx device: CPU
     row          col 
        ----------    
        |        |    
        |      2 |-->  a
        |        |    
        |      2 |-->  b
        |        |    
        |      4 |<--* c
        |        |    
        ----------    



In [39]:
uTsym_dag.print_blocks()

-------- start of print ---------
Tensor name: uTsym
braket_form : False
is_diag    : False
[OVERALL] contiguous : True
BLOCK [#0]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
        ----------
        |        |
        |      1 |-->  [0] U1(1)
        |        |
        |      1 |-->  [0] U1(1)
        |        |
        |      1 |<--* [0] U1(2)
        |        |
        ----------

Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,1)
[[[0.00000e+00 ]]]

BLOCK [#1]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
        ----------
        |        |
        |      1 |-->  [0] U1(1)
        |        |
        |      1 |-->  [1] U1(-1)
        |        |
        |      2 |<--* [1] U1(0)
        |        |
        ----------

Total elem: 2
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,2)
[[[0.00000e+00 0.00000e+00 ]]]

BLOCK [#2]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
        ----------
        |        |
  

In [40]:
print(uTsym)

-------- start of print ---------
Tensor name: uTsym
braket_form : False
is_diag    : False
[OVERALL] contiguous : True
BLOCK [#0]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
        ----------
        |        |
        |      1 |<--* [0] U1(1)
        |        |
        |      1 |<--* [0] U1(1)
        |        |
        |      1 |-->  [0] U1(2)
        |        |
        ----------

Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,1)
[[[0.00000e+00 ]]]

BLOCK [#1]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
        ----------
        |        |
        |      1 |<--* [0] U1(1)
        |        |
        |      1 |<--* [1] U1(-1)
        |        |
        |      2 |-->  [1] U1(0)
        |        |
        ----------

Total elem: 2
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,2)
[[[0.00000e+00 0.00000e+00 ]]]

BLOCK [#2]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
        ----------
        |        |
  

### Symmetric UniTensor

#### Creating a symmetric UniTensor

In [3]:
# checked
# Create a symmetric \lstinline{UniTensor} using bonds with quantum numbers
# Define the bonds with quantum numbers
bond1 = cytnx.Bond(cytnx.BD_IN,\
                [cytnx.Qs(1)>>1, cytnx.Qs(-1)>>1],[cytnx.Symmetry.U1()])
bond2 = cytnx.Bond(cytnx.BD_IN,\
                [cytnx.Qs(1)>>1, cytnx.Qs(-1)>>1],[cytnx.Symmetry.U1()])
bond3 = cytnx.Bond(cytnx.BD_OUT,\
                [cytnx.Qs(2)>>1, cytnx.Qs(0)>>2, cytnx.Qs(-2)>>1],\
                [cytnx.Symmetry.U1()])
# Create a symmetric UniTensor with these bonds
uTsym = cytnx.UniTensor([bond1, bond2, bond3]).relabels(["a","b","c"])\
                        .set_name("uTsym")
uTsym.print_diagram()

-----------------------
tensor Name : uTsym
tensor Rank : 3
contiguous  : True
valid blocks : 4
is diag   : False
on device   : cytnx device: CPU
      row           col 
         -----------    
         |         |    
   a  -->| 2     4 |-->  c
         |         |    
   b  -->| 2       |        
         |         |    
         -----------    



<img src="figs/uniTensor_symmetry_U1.png" alt="drawing" width="400px"/>

In [42]:
uTsym.print_blocks()

-------- start of print ---------
Tensor name: uTsym
braket_form : True
is_diag    : False
[OVERALL] contiguous : True
BLOCK [#0]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
                 -----------
                 |         |
   [0] U1(1)  -->| 1     1 |-->  [0] U1(2)
                 |         |
   [0] U1(1)  -->| 1       |
                 |         |
                 -----------

Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,1)
[[[0.00000e+00 ]]]

BLOCK [#1]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
                  -----------
                  |         |
   [0] U1(1)   -->| 1     2 |-->  [1] U1(0)
                  |         |
   [1] U1(-1)  -->| 1       |
                  |         |
                  -----------

Total elem: 2
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,2)
[[[0.00000e+00 0.00000e+00 ]]]

BLOCK [#2]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
                  ---------

#### Accessing elements of a symmetric tensor

In [43]:
# print(uTsym.at([0,0,1]).value)

In [44]:
print(uTsym.at([0,0,0]).exists()) # Output: True
print(uTsym.at([0,0,0]).value)
print(uTsym.at([0,0,1]).exists()) # Output: False

True
0.0
False


#### Accessing blocks of a symmetric tensor

In [45]:
# Get a block by its Qn indices
# The first argument specifies the labels of the indices,
# and the second argument the corresponding Qn indices
B1 = uTsym.get_block_(["a","b","c"],[0,1,1])
# Get a block by its block index
B2 = uTsym.get_block_(1)

In [46]:
# Get all the blocks in a UniTensor
Blks = uTsym.get_blocks_()
# Print the number of blocks
print(len(Blks))    # Output: 4
# Print each block
print( *Blks )

4

Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,1)
[[[0.00000e+00 ]]]


 
Total elem: 2
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,2)
[[[0.00000e+00 0.00000e+00 ]]]


 
Total elem: 2
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,2)
[[[0.00000e+00 0.00000e+00 ]]]


 
Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,1)
[[[0.00000e+00 ]]]





#### Putting a block

In [47]:
# Get a block as a Tensor object by Qn indices
B1 = uTsym.get_block([0,1,1])
# Manipulate the Tensor
B1[0,0,0] = 1.
# Put the Tensor to a block by Qn indices
uTsym.put_block(B1,[0,1,1])
# Put the Tensor to a block by block index
uTsym.put_block(B1,1)
print(uTsym.get_block(1))


Total elem: 2
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,2)
[[[1.00000e+00 0.00000e+00 ]]]





### Change the bond direction

In [48]:
# Create bonds
bond1 = cytnx.Bond(1,cytnx.BD_IN)
bond2 = bond1.redirect()
# Create a symmetric UniTensor with these bonds
uTc = cytnx.UniTensor([bond1,bond2], dtype=cytnx.Type.ComplexDouble)\
                      .relabels(["a", "b"]).set_name("uTc")
# Set an element
uTc.at([0,0]).value = 1.+2.j
# Change the direction of the bonds
uTtransp = uTc.Transpose().set_name("uTc transpose")
# Conjugate the elements
uTconj = uTc.Conj().set_name("uTc conjugate")
# Take the Hermitian conjugate
uTdag = uTc.Dagger().set_name("uTc Hermitian conjugate")
# Equivalently:
uTdag2 = uTc.Conj().Transpose().set_name("uTc conjugate transpose")
print(uTc)      # Part of output: [[1.00000e+00+2.00000e+00j ]]
print(uTconj)   # Part of output: [[1.00000e+00-2.00000e+00j ]]
print(uTtransp) # Part of output: [[1.00000e+00+2.00000e+00j ]]
print(uTdag)    # Part of output: [[1.00000e+00-2.00000e+00j ]]

-------- start of print ---------
Tensor name: uTc
braket_form : True
is_diag    : False
contiguous : True

Total elem: 1
type  : Complex Double (Complex Float64)
cytnx device: CPU
Shape : (1,1)
[[1.00000e+00+2.00000e+00j ]]




-------- start of print ---------
Tensor name: uTc conjugate
braket_form : True
is_diag    : False
contiguous : True

Total elem: 1
type  : Complex Double (Complex Float64)
cytnx device: CPU
Shape : (1,1)
[[1.00000e+00-2.00000e+00j ]]




-------- start of print ---------
Tensor name: uTc transpose
braket_form : False
is_diag    : False
contiguous : True

Total elem: 1
type  : Complex Double (Complex Float64)
cytnx device: CPU
Shape : (1,1)
[[1.00000e+00+2.00000e+00j ]]




-------- start of print ---------
Tensor name: uTc Hermitian conjugate
braket_form : False
is_diag    : False
contiguous : True

Total elem: 1
type  : Complex Double (Complex Float64)
cytnx device: CPU
Shape : (1,1)
[[1.00000e+00-2.00000e+00j ]]






### Conversion between UniTensor with and without symmetries

In [49]:
# Create sigma matrices as non-symmetric UniTensors
Sp = cytnx.UniTensor.zeros([2,2]) # sigma^+
Sp[0, 1] = +1.
Sm = cytnx.UniTensor.zeros([2,2]) # sigma^-
Sm[1, 0] = +1.
Sz = cytnx.UniTensor.zeros([2,2]) # sigma^z
Sz[0, 0] = +1.
Sz[1, 1] = -1.
# Create two copies of all sigma matrices for the two sites
Sp1 = Sp.clone().set_name("Sp1").relabels(["i1","j1"])
Sp2 = Sp.clone().set_name("Sp2").relabels(["i2","j2"])
Sm1 = Sm.clone().set_name("Sm1").relabels(["i1","j1"])
Sm2 = Sm.clone().set_name("Sm2").relabels(["i2","j2"])
Sz1 = Sz.clone().set_name("Sz1").relabels(["i1","j1"])
Sz2 = Sz.clone().set_name("Sz2").relabels(["i2","j2"])
# Kronecker product of two sigma matrices at different sites
Sp1Sm2 = cytnx.Contract(Sp1,Sm2).set_name("Sp1Sm2")
Sp1Sm2.permute_(["i1","i2","j1","j2"])
Sm1Sp2 = cytnx.Contract(Sm1,Sp2).set_name("Sm1Sp2")
Sm1Sp2.permute_(["i1","i2","j1","j2"])
Sz1Sz2 = cytnx.Contract(Sz1,Sz2).set_name("Sz1Sz2")
Sz1Sz2.permute_(["i1","i2","j1","j2"])
# Create Hamiltonian as non-symmetric UniTensor
H = Sp1Sm2 + Sm1Sp2 + Sz1Sz2
H.set_name("H non-Symmetric").relabels_(["i1","i2","j1","j2"])
# Print in matrix form to check
print(H.reshape(4,4))

-------- start of print ---------
Tensor name: 
is_diag    : False
contiguous : True

Total elem: 16
type  : Double (Float64)
cytnx device: CPU
Shape : (4,4)
[[1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 ]
 [0.00000e+00 -1.00000e+00 1.00000e+00 0.00000e+00 ]
 [0.00000e+00 1.00000e+00 -1.00000e+00 0.00000e+00 ]
 [0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 ]]






In [50]:
# Supposes H is a UniTensor that is already created as in (*\color{codegreen}\cref{code:create_Hamiltonian_dense}*)
# Create bonds with U(1) symmetry
bi = cytnx.Bond(cytnx.BD_IN, [cytnx.Qs(+1)>>1, cytnx.Qs(-1)>>1],\
                [cytnx.Symmetry.U1()])
bo = cytnx.Bond(cytnx.BD_OUT, [cytnx.Qs(+1)>>1, cytnx.Qs(-1)>>1],\
                [cytnx.Symmetry.U1()])
# Initialize U(1) symmetric UniTensor
H_sym = cytnx.UniTensor([bi,bi,bo,bo]).set_name("H symmetric")\
                        .relabels(["i1","i2","j1","j2"])
# Copy data from the non-symmetric UniTensor H
H_sym.convert_from(H)
print(H_sym) # Check the blocks

-------- start of print ---------
Tensor name: H symmetric
braket_form : True
is_diag    : False
[OVERALL] contiguous : True
BLOCK [#0]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
                 -----------
                 |         |
   [0] U1(1)  -->| 1     1 |-->  [0] U1(1)
                 |         |
   [0] U1(1)  -->| 1     1 |-->  [0] U1(1)
                 |         |
                 -----------

Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,1,1)
[[[[1.00000e+00 ]]]]

BLOCK [#1]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
                  -----------
                  |         |
   [0] U1(1)   -->| 1     1 |-->  [0] U1(1)
                  |         |
   [1] U1(-1)  -->| 1     1 |-->  [1] U1(-1)
                  |         |
                  -----------

Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,1,1)
[[[[-1.00000e+00 ]]]]

BLOCK [#2]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symm

In [51]:
# Supposes H_sym is a symmetric UniTensor that is already created
# as in (*\color{codegreen}\cref{code:convert_dense_to_sym}*)
# Create bonds with U(1) symmetry
bi = cytnx.Bond(cytnx.BD_IN, [cytnx.Qs(+1)>>1, cytnx.Qs(-1)>>1],\
                [cytnx.Symmetry.U1()])
bo = cytnx.Bond(cytnx.BD_OUT, [cytnx.Qs(+1)>>1, cytnx.Qs(-1)>>1],\
                [cytnx.Symmetry.U1()])
# Bond between sigma^+ and sigma^- needs to carry a quantum number
bqo = cytnx.Bond(cytnx.BD_OUT, [cytnx.Qs(+2)>>1],\
                [cytnx.Symmetry.U1()])
bqi = cytnx.Bond(cytnx.BD_IN, [cytnx.Qs(+2)>>1],\
                [cytnx.Symmetry.U1()])
# Create sigma matrices as symmetric UniTensors
Sp = cytnx.UniTensor([bi,bo,bqo])   # sigma^+
Sp.at([0, 1, 0]).value = +1.
Sm = cytnx.UniTensor([bqi,bi,bo])   # sigma^-
Sm.at([0, 1, 0]).value = +1.
Sz = cytnx.UniTensor([bi,bo])       # sigma^z
Sz.at([0, 0]).value = +1.
Sz.at([1, 1]).value = -1.
# Create two copies of all sigma matrices for the two sites
Sp1 = Sp.clone().set_name("Sp1").relabels(["i1","j1","q"])
Sp2 = Sp.clone().set_name("Sp2").relabels(["i2","j2","q"])
Sm1 = Sm.clone().set_name("Sm1").relabels(["q","i1","j1"])
Sm2 = Sm.clone().set_name("Sm2").relabels(["q","i2","j2"])
Sz1 = Sz.clone().set_name("Sz1").relabels(["i1","j1"])
Sz2 = Sz.clone().set_name("Sz2").relabels(["i2","j2"])
# Product of two sigma matrices at different sites;
# labels "q" are summed over
Sp1Sm2 = cytnx.Contract(Sp1,Sm2).set_name("Sp1Sm2")
Sp1Sm2.permute_(["i1","i2","j1","j2"])
Sm1Sp2 = cytnx.Contract(Sm1,Sp2).set_name("Sm1Sp2")
Sm1Sp2.permute_(["i1","i2","j1","j2"])
Sz1Sz2 = cytnx.Contract(Sz1,Sz2).set_name("Sz1Sz2")
Sz1Sz2.permute_(["i1","i2","j1","j2"])
# Create Hamiltonian as symmetric UniTensor
H2_sym = Sp1Sm2 + Sm1Sp2 + Sz1Sz2
H2_sym.set_name("H2 symmetric").relabels_(["i1","i2","j1","j2"])
# Check the blocks
print(H2_sym - H_sym) # Output: all elements are zero

-------- start of print ---------
Tensor name: 
braket_form : True
is_diag    : False
[OVERALL] contiguous : False
BLOCK [#0]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
                 -----------
                 |         |
   [0] U1(1)  -->| 1     1 |-->  [0] U1(1)
                 |         |
   [0] U1(1)  -->| 1     1 |-->  [0] U1(1)
                 |         |
                 -----------

Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,1,1)
[[[[0.00000e+00 ]]]]

BLOCK [#1]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
                  -----------
                  |         |
   [0] U1(1)   -->| 1     1 |-->  [0] U1(1)
                  |         |
   [1] U1(-1)  -->| 1     1 |-->  [1] U1(-1)
                  |         |
                  -----------

Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1,1,1,1)
[[[[0.00000e+00 ]]]]

BLOCK [#2]
 |- []   : Qn index 
 |- Sym(): Qnum of correspond symmetry
      

## Tensor contraction

## Network

### Creating a Network

In [52]:
# Define a Network object for the contraction task in (*\color{codegreen}\cref{fig:MMM}*)
net = cytnx.Network()
net.FromString(["M1: i, j",\
                "M2: j, k",\
                "M3: k, l",\
                "TOUT: i; l",\
                "ORDER: ((M1,M2),M3)"])
print(net)

==== Network ====
[x] M1 : i j 
[x] M2 : j k 
[x] M3 : k l 
TOUT : i ; l 
ORDER : ((M1,M2),M3)



### Putting UniTensors and performing a contraction

In [53]:
# Supposes net is a Network that is already created as in (*\color{codegreen}\cref{code:NetworkMMM}*)
# Create UniTensors that are going to be contracted
A = cytnx.UniTensor.ones([2,2]).relabels(["a","b"]).set_name("A")
B = cytnx.UniTensor.ones([2,2]).relabels(["a","b"]).set_name("B")
C = cytnx.UniTensor.ones([2,2]).relabels(["a","b"]).set_name("C")
# Put UniTensors into a Network object
net.PutUniTensor("M1", A, ["a","b"])
net.PutUniTensor("M2", B, ["a","b"])
net.PutUniTensor("M3", C, ["a","b"])
# Launch the contraction
Tout = net.Launch()
Tout.print_diagram()
print(Tout)

-----------------------
tensor Name : 
tensor Rank : 2
block_form  : False
is_diag     : False
on device   : cytnx device: CPU
          ---------     
         /         \    
   i ____| 2     2 |____ l
         \         /    
          ---------     
-------- start of print ---------
Tensor name: 
is_diag    : False
contiguous : True

Total elem: 4
type  : Double (Float64)
cytnx device: CPU
Shape : (2,2)
[[4.00000e+00 4.00000e+00 ]
 [4.00000e+00 4.00000e+00 ]]






### Setting the contraction order

In [54]:
# Supposes net is a Network that is already created as in (*\color{codegreen}\cref{code:NetworkMMM}*)
# and all tensors are loaded (put) as in (*\color{codegreen}\cref{code:PutTensor}*)
# Calculate the optimal contraction order for the loaded tensors
net.setOrder(optimal=True)
# Set a specific contraction order
net.setOrder(optimal=False, contract_order='(M2,(M1,M3))')
# Print the current contraction order
print(net.getOrder()) # Output: (M2,(M1,M3))

(M2,(M1,M3))


## Tensor decomposition and linear algebra

### Mapping from tensor to matrix

In [55]:
# checked, modified
# Matrix form of a \lstinline{UniTensor}}
# Create a rank-3 tensor with shape [2,2,6]
M = cytnx.UniTensor.arange(2*2*6).reshape(2,2,6)\
                           .relabel(["a","b","c"]).set_name("M")
# 1. Permute the indices to ("c","a","b") ordering,
M.permute_(["c","a","b"])
# 2. Set rowrank=1, such that "c" is the row index,
# and "a", "b" are the column indices.
M.set_rowrank_(1)
print(M.rowrank()) # Output: 1
M.print_diagram()

1
-----------------------
tensor Name : M
tensor Rank : 3
block_form  : False
is_diag     : False
on device   : cytnx device: CPU
          ---------     
         /         \    
   c ____| 6     2 |____ a
         |         |    
         |       2 |____ b
         \         /    
          ---------     


### Singular value decomposition

In [56]:
# Supposes M is a UniTensor that is already created as in (*\color{codegreen}\cref{code:matrixForm}*)
# Perform SVD on M
S, U, Vdag = cytnx.linalg.Svd(M)
# Compute the product of U*S*Vdag
Msvd = cytnx.Contracts([U,S,Vdag])
Msvd.permute_(M.labels())
# Check that M = U*S*Vdag
Diff = M - Msvd
print(Diff.Norm()) # Output: 2.79115e-14


Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1)
[2.75999e-14 ]





#### With truncation

In [57]:
# Supposes M is a UniTensor that is already created as in (*\color{codegreen}\cref{code:matrixForm}*)
# Perform SVD without truncation and return only the singular values
S_full = cytnx.linalg.Svd (M, is_UvT=False)
# Perform SVD with truncation
S_trunc, U, Vdag, s_err = cytnx.linalg.Svd_truncate\
                      (M, keepdim=3, err=1e-10, return_err=1)
# Compare the full sigular values with the kept and the truncated ones
print(S_full)   # [6.56235e+01 4.18988e+00 2.92323e-15 8.22506e-16 ]
print(S_trunc)  # [6.56235e+01 4.18988e+00 ]
print(s_err)    # [2.92323e-15 ]

-------- start of print ---------
Tensor name: 
is_diag    : True
contiguous : True

Total elem: 4
type  : Double (Float64)
cytnx device: CPU
Shape : (4)
[6.56235e+01 4.18988e+00 3.81402e-15 9.85422e-16 ]



[]
-------- start of print ---------
Tensor name: 
is_diag    : True
contiguous : True

Total elem: 2
type  : Double (Float64)
cytnx device: CPU
Shape : (2)
[6.56235e+01 4.18988e+00 ]




-------- start of print ---------
Tensor name: 
is_diag    : False
contiguous : True

Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1)
[3.81402e-15 ]






### Eigenvalue decomposition

In [4]:
# Create a randomly initialized Hermitian matrix
uT = cytnx.UniTensor.uniform([4,4], low=-1., high=1.)
uT = uT + uT.Conj().Transpose()
uT = uT.relabels(["a","b"]).set_name("uT").set_rowrank(1)
# Eigenvalue decomposition
eigvals, V = cytnx.linalg.Eigh(uT)
Vdag = V.Conj().Transpose()
# Set labels
V.relabels_(["a","_aux_L"]).set_name("V")
eigvals.relabels_(["_aux_L","_aux_R"]).set_name("eigvals")
Vdag.relabels_(["_aux_R","b"]).set_name("Vdag")
# Compare uT with V*eigvals*Vdag
Diff = uT - cytnx.Contracts([V,eigvals,Vdag])
print(Diff.Norm()) # Output: 1.59153e-15
# Create identity matrix
eye = cytnx.UniTensor.zeros([4,4])
for i in range(4):
    eye[i,i] = 1
eye.relabels_(["_aux_L","_aux_R"]).set_name("eye")
# Compare eye with V*eye*Vdag
Diff = eye - cytnx.Contracts([V,eye,Vdag])
print(Diff.Norm()) # Output: 6.09676e-16


Setting root pointers from ((T0,T1),T2)
Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1)
[4.49625e-15 ]




Setting root pointers from ((T0,T1),T2)
Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1)
[1.53966e-15 ]





In [5]:
V.print_diagram()

-----------------------
tensor Name : V
tensor Rank : 2
block_form  : False
is_diag     : False
on device   : cytnx device: CPU
          ---------     
         /         \    
   a ____| 4     4 |____ _aux_L
         \         /    
          ---------     


### QR decomposition

In [59]:
# Initial matrix
uT = cytnx.UniTensor.arange(5*4).reshape(5,4).relabels(["a","d"])\
                          .set_name("uT").set_rowrank(1)
# QR decomposition
Q, R = cytnx.linalg.Qr(uT)
# Compare uT with Q*R
Diff = uT - cytnx.Contract(Q,R)
print(Diff.Norm()) # Output: 1.14885e-14
# Check properties of Q,R
print(R) # upper triangular
QT = Q.Transpose().relabels_(["a","b"])
Q.relabels_(["b","c"])
eye = cytnx.UniTensor.zeros([4,4])
for i in range(4):
    eye[i,i] = 1
Diff = cytnx.Contract(QT, Q) - eye
print(Diff.Norm()) # Output: 6.13418e-16


Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1)
[1.14541e-14 ]



-------- start of print ---------
Tensor name: 
is_diag    : False
contiguous : True

Total elem: 16
type  : Double (Float64)
cytnx device: CPU
Shape : (4,4)
[[-2.19089e+01 -2.37346e+01 -2.55604e+01 -2.73861e+01 ]
 [0.00000e+00 -1.29099e+00 -2.58199e+00 -3.87298e+00 ]
 [0.00000e+00 0.00000e+00 3.03515e-15 5.66076e-15 ]
 [0.00000e+00 0.00000e+00 0.00000e+00 -2.82431e-16 ]]





Total elem: 1
type  : Double (Float64)
cytnx device: CPU
Shape : (1)
[5.73485e-16 ]





## Device and GPU support

In [61]:
print(cytnx.Device.Ncpus)
print(cytnx.Device.Ngpus)

1
0
